In [ ]:
import numpy as np
import pandas as pd
from patsy import dmatrix
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge

# Sample data
x = np.linspace(0, 10, 100)
df = pd.DataFrame({'x': x})

# Generate B-spline basis (not penalized yet)
spline_basis = dmatrix("bs(x, df=10, degree=3, include_intercept=False)", data=df)
basis_array = np.asarray(spline_basis)

# Target variable: make a synthetic one
y = np.sin(x) + 0.1 * np.random.randn(len(x))

# Ridge regression on spline basis (P-spline equivalent)
ridge = Ridge(alpha=5.0)  # alpha controls the amount of penalization
ridge.fit(spline_basis, y)

# Predict
y_pred = ridge.predict(spline_basis)

# Plot
plt.plot(x, y, label='Data')
plt.plot(x, y_pred, label='P-spline (Ridge on B-splines)')
plt.legend()
plt.show()


In [ ]:
# Plot each basis function
plt.figure(figsize=(10, 6))
for i in range(basis_array.shape[1]):
    plt.plot(x, basis_array[:, i], label=f"Basis {i+1}")

plt.title("B-spline Basis Functions (from patsy.bs)")
plt.xlabel("x")
plt.ylabel("Basis Function Value")
plt.legend(ncol=2, fontsize=9)
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from patsy import dmatrix

# Input data
x = np.linspace(0, 10, 200)
df = pd.DataFrame({'x': x})

# Generate B-spline basis
basis = dmatrix("bs(x, df=10, degree=3, include_intercept=True)", data=df)
B = np.asarray(basis)

# Plot B-spline basis functions
plt.figure(figsize=(10, 6))
for i in range(B.shape[1]):
    plt.plot(x, B[:, i], label=f'B{i}')
plt.title("B-spline Basis Functions")
plt.xlabel("x")
plt.ylabel("Basis Value")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# True underlying function
true_y = np.sin(x) + 0.5 * np.cos(1.5 * x)
# Noisy observations
y_obs = true_y + np.random.normal(0, 0.2, size=x.shape[0])


In [ ]:
import pymc as pm
import arviz as az

# Number of basis functions
n_basis = B.shape[1]

with pm.Model() as model:
    # Prior: smoothness via 2nd-order random walk
    sigma_rw = pm.HalfNormal("sigma_rw", 1.0)

    # 2nd-order random walk prior for spline coefficients
    beta = pm.GaussianRandomWalk("beta", sigma=sigma_rw, shape=n_basis)

    # Expected value
    mu = pm.math.dot(B, beta)

    # Observation noise
    sigma = pm.HalfNormal("sigma", 1.0)
    y = pm.Normal("y", mu=mu, sigma=sigma, observed=y_obs)

    # Sample from posterior
    trace = pm.sample(1000, tune=1000, target_accept=0.95, return_inferencedata=True)


In [ ]:
# Posterior samples of fitted curve
posterior_mu = trace.posterior["beta"].stack(draws=("chain", "draw"))
mu_samples = B @ posterior_mu.values

# Compute mean and HDI
mu_mean = mu_samples.mean(axis=1)
mu_hdi = az.hdi(mu_samples.T, hdi_prob=0.94)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(x, y_obs, 'o', alpha=0.5, label='Observed')
plt.plot(x, true_y, label='True function', linestyle='--')
plt.plot(x, mu_mean, label='Posterior mean', color='k')
plt.fill_between(x, mu_hdi[:, 0], mu_hdi[:, 1], color='gray', alpha=0.3, label='94% HDI')
plt.legend()
plt.title("Bayesian P-spline Fit with PyMC")
plt.xlabel("x")
plt.ylabel("y")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import pymc as pm
from patsy import dmatrix
import matplotlib.pyplot as plt


In [ ]:
np.random.seed(42)
n = 100
x = np.linspace(0, 10, n)
y_true = np.sin(x)
y = y_true + np.random.normal(scale=0.3, size=n)

data = pd.DataFrame({'x': x, 'y': y})


In [ ]:
# Create B-spline basis (degree=3 cubic, df=10, include intercept)
spline_basis = dmatrix("bs(x, df=10, degree=3, include_intercept=True)", data)
X_spline = np.asarray(spline_basis)
n_basis = X_spline.shape[1]


In [ ]:
with pm.Model() as model:
    # Prior on spline coefficients with smoothing via a random walk (difference penalty)
    sigma_beta = pm.HalfNormal("sigma_beta", sigma=1)
    
    # Use a random walk (order 2) for smoothness - typical for P-splines
    beta = pm.GaussianRandomWalk("beta", sigma=sigma_beta, shape=n_basis)
    
    # Linear combination of spline basis and coefficients
    mu = pm.math.dot(X_spline, beta)
    
    # Likelihood
    sigma = pm.HalfNormal("sigma", sigma=1)
    y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=y)
    
    # Inference
    trace = pm.sample(1000, tune=1000, target_accept=0.9)



In [ ]:
with model:
    posterior_predictive = pm.sample_posterior_predictive(trace)

y_pred = posterior_predictive.posterior_predictive["y_obs"].values

# Plot results
plt.figure(figsize=(10, 6))
plt.scatter(x, y, label="Observed")
plt.plot(x, y_true, label="True", color='black', linestyle='--')
plt.plot(x, y_pred.mean(axis=(0, 1)), label="Fitted", color='red')
plt.fill_between(
    x,
    np.percentile(y_pred, 5, axis=(0, 1)),
    np.percentile(y_pred, 95, axis=(0, 1)),
    color="red", alpha=0.3
)
plt.legend()
plt.title("P-spline fit using PyMC and Patsy")
plt.xlabel("x")
plt.ylabel("y")
plt.show()
